In [1]:
# Autoreload
%load_ext autoreload
%autoreload 2

In [2]:
import wandb
import os
os.environ['WANDB_NOTEBOOK_NAME'] = 'main.ipynb'
wandb.init(project="Mesh-Learning")

wandb: Currently logged in as: robinpourtaud. Use `wandb login --relogin` to force relogin


In [3]:
import torch
from torch_geometric.datasets import MNISTSuperpixels
from torch_geometric import transforms as T
def pre_transform(data):
    edge_index = data.edge_index
    edges_mask = (data.x[edge_index[0]] >= 0.65) & (data.x[edge_index[1]] >= 0.65)
    edges_to_keep = torch.nonzero(edges_mask, as_tuple=True)[0]
    data.edge_index = edge_index[:, edges_to_keep]
    data.x = data.pos
    data = T.RemoveIsolatedNodes()(data)
    return data
data = MNISTSuperpixels("data/MNISTSuperpixels", pre_transform=pre_transform)

Extracting data/MNISTSuperpixels/raw/MNISTSuperpixels.zip
Processing...
Done!


In [ ]:
# Import necessary libraries
import torch
from torch_geometric.utils import to_networkx
import networkx as nx
import matplotlib.pyplot as plt

# Load model
encoder = GenericEncoder(GCNConv, 2, 32, 16)
decoder = GCNConv(16, 10)
model = VGAELightning(encoder, decoder, train_dataset, lr=0.01, batch_size=64)
model.load_state_dict(torch.load("model.pt"))
model.eval() # Set model to evaluation mode

# Sample from the latent space
z = torch.randn(1, model.encoder.latent_dim) 

# Use the decoder to generate a graph
generated_graph = model.decoder(z)

# Convert the generated graph to networkx for easy visualization
nx_graph = to_networkx(generated_graph).to_undirected()

# Visualize the generated graph
nx.draw(nx_graph, with_labels=True)
plt.show()
